# AutoGen AgentChat – Finance Mini‑Workshop
Welcome! This notebook walks you through three incremental steps:
1. **A single agent** – like hiring one junior analyst.
2. **Adding tools** – giving that analyst a Bloomberg terminal.
3. **Forming a team** – multiple specialists who chat to solve a task.

Run each cell and read the comments. ⚠️ Make sure you have an OpenAI‑compatible key in your environment:
```bash
export OPENAI_API_KEY="sk‑…"
```


## Setup
Install the libraries if you haven’t already (uncomment the pip line if running for the first time).

In [ ]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]" --quiet
! pip install -U "autogen-agentchat" "autogen-ext[openai]" pandas python-dotenv

In [30]:
API_KEY=''

## 1️⃣  A single agent
We start with one chat‑based **AssistantAgent** called `Greeter`. No tools yet – it just chats using the OpenAI model.

In [31]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import asyncio

# Create a model client (replace model if you prefer)
model_client = OpenAIChatCompletionClient(model="gpt-4o",api_key=API_KEY)

# --- define the agent -------------------------------------------------
greeter = AssistantAgent(
    name="Greeter",
    model_client=model_client,
    system_message="You are Greeter, a friendly finance assistant. Keep replies short."
)

# --- run a single message --------------------------------------------
async def single_demo():
    task = "Give me a one-sentence summary of Apple's stock performance."
    await Console(greeter.run_stream(task=task))

await single_demo()

---------- TextMessage (user) ----------
Give me a one-sentence summary of Apple's stock performance.
---------- TextMessage (Greeter) ----------
Apple's stock performance has generally shown strong growth, achieving new highs in recent years despite some volatility.


## 2️⃣  Introducing tools
Agents become far more useful when you give them functions they can call. We’ll define a *very* simple price lookup tool and attach it to a new agent.

In [32]:
import random
from typing import List


# ---- TOOL DEFINITIONS ------------------------------------------------
def get_stock_price(ticker: str) -> float:
    """Return a *mock* price for the given ticker (USD)."""
    price_table = {"AAPL": 180.15, "MSFT": 410.22, "GOOG": 156.42}
    return price_table.get(ticker.upper(), random.uniform(50, 500))

# ---- AGENT WITH TOOL -------------------------------------------------
price_agent = AssistantAgent(
    name="PriceAgent",
    model_client=model_client,
    system_message=(
        "You are PriceAgent. Use the get_stock_price function to answer price questions."
    ),
    tools=[get_stock_price],   # auto‑wrapped as FunctionTool
)

# ---- run a demo ------------------------------------------------------
async def tool_demo():
    task = "What is Apple's current price and how does it compare to yesterday's $192 close?"
    await Console(price_agent.run_stream(task=task))

await tool_demo()


---------- TextMessage (user) ----------
What is Apple's current price and how does it compare to yesterday's $192 close?
---------- ToolCallRequestEvent (PriceAgent) ----------
[FunctionCall(id='call_dGb9llbFpfF3b1BgIzaZeeY0', arguments='{"ticker":"AAPL"}', name='get_stock_price')]
---------- ToolCallExecutionEvent (PriceAgent) ----------
[FunctionExecutionResult(content='180.15', name='get_stock_price', call_id='call_dGb9llbFpfF3b1BgIzaZeeY0', is_error=False)]
---------- ToolCallSummaryMessage (PriceAgent) ----------
180.15


## 3️⃣  Building a team
Now let’s put several specialists around a *virtual* desk. They will chat in round‑robin fashion until a termination condition is met.

In [33]:
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from datetime import datetime

# ---- another tool for headlines -------------------------------------
def get_news_headlines(ticker: str) -> List[str]:
    """Return *mock* list of news headlines for the ticker."""
    sample_news = {
        "AAPL": [
            "Apple supplier faces production hiccup, cutting quarterly output 8%",
            "Analysts debate impact of slower iPhone upgrade cycle",
            "EU launches investigation into iOS anti‑steering rules",
        ]
    }
    return sample_news.get(ticker.upper(), ["No material headlines found."])

# ---- helper to create agents ----------------------------------------
def make_agent(name: str, role: str):
    return AssistantAgent(
        name=name,
        model_client=model_client,
        system_message=f"You are {name}. {role} Keep replies concise.",
        tools=[get_stock_price, get_news_headlines],
    )

price_watcher = make_agent(
    "PriceWatcher",
    "Track AAPL and explain any intraday move >5% using headlines."
)
news_summarizer = make_agent(
    "NewsSummarizer",
    "Condense raw headlines into two sentences."
)
risk_officer = make_agent(
    "RiskOfficer",
    "Assess how the price move affects portfolio VaR; flag if new VaR > $5m."
)
trader_bot = make_agent(
    "TraderBot",
    "Suggest one actionable trade (Buy/Sell/Hold) with a one-line rationale."
)

team = RoundRobinGroupChat(
    participants=[price_watcher, news_summarizer, risk_officer, trader_bot],
    termination_condition=MaxMessageTermination(6),
)

# ---- trigger an alert ------------------------------------------------
PORTFOLIO = {"AAPL": 100_000_000}  # USD notional
yesterday_close = 192.00
now_price = get_stock_price("AAPL")
pct_move = (now_price - yesterday_close) / yesterday_close * 100

async def team_demo():
    if pct_move < -5:
        alert = (
            f"ALERT {datetime.now().strftime('%H:%M')} – AAPL down {pct_move:.1f}% intraday\n"
            f"Position: ${PORTFOLIO['AAPL']:,.0f}. VaR limit $5m.\n"
            "Please analyse cause, risk and propose one action."
        )
        await Console(team.run_stream(task=alert))
    else:
        print(f"No alert – AAPL move only {pct_move:.2f}% (<5%). Price ${now_price:.2f}")

await team_demo()


---------- TextMessage (user) ----------
ALERT 08:50 – AAPL down -6.2% intraday
Position: $100,000,000. VaR limit $5m.
Please analyse cause, risk and propose one action.
---------- ToolCallRequestEvent (PriceWatcher) ----------
[FunctionCall(id='call_RL2x6YqPX0ljBNgqzApovPHE', arguments='{"ticker": "AAPL"}', name='get_stock_price'), FunctionCall(id='call_Z5guApyhePHMwIcoLCTRRJgX', arguments='{"ticker": "AAPL"}', name='get_news_headlines')]
---------- ToolCallExecutionEvent (PriceWatcher) ----------
[FunctionExecutionResult(content='180.15', name='get_stock_price', call_id='call_RL2x6YqPX0ljBNgqzApovPHE', is_error=False), FunctionExecutionResult(content="['Apple supplier faces production hiccup, cutting quarterly output 8%', 'Analysts debate impact of slower iPhone upgrade cycle', 'EU launches investigation into iOS anti‑steering rules']", name='get_news_headlines', call_id='call_Z5guApyhePHMwIcoLCTRRJgX', is_error=False)]
---------- ToolCallSummaryMessage (PriceWatcher) ----------
180.

### 🎉 You’ve just seen…
* A lone agent chatting.
* A tool‑enabled agent.
* A multi‑agent team solving a task.

From here you can:
* Swap `RoundRobinGroupChat` for `SelectorGroupChat` to route tasks dynamically.
* Replace the mock tools with real data feeds (e.g., Bloomberg, SQL, web APIs).
* Add logging, memory, or a custom termination strategy.

Happy experimenting! 🚀